In [15]:
!pip install pandas scikit-learn tensorflow --upgrade keras nltk gensim transformers

  Using cached numpy-1.23.5-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
ERROR: Cannot uninstall numpy 1.24.3, RECORD file not found. You might be able to recover from this via: 'pip install --force-reinstall --no-deps numpy==1.24.3'.


In [16]:
import os
import re

import gensim
import numpy as np
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
from nltk.tokenize.treebank import TreebankWordDetokenizer
import tensorflow as tf


In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/alexa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
df = pd.read_csv("data/amazon.csv", delimiter=";")
df.drop(columns=[df.columns[0], "Разница в долларах", "Дельта в процентах"], axis=1, inplace=True)
for root, _, files in os.walk("data"):
    for filename in files:
        temp_df = pd.read_csv(os.path.join(root, filename), delimiter=";")
        temp_df.drop(columns=[temp_df.columns[0], "Разница в долларах", "Дельта в процентах"], axis=1, inplace=True, errors="ignore")
        print(filename, temp_df.head())
        df = pd.concat([df, temp_df], axis=0, sort=False)

df.info()
df.drop(columns=[df.columns[5]])

northrop.csv          Дата     Время     Цена до  Цена после   
0  2022-10-14  14:10:32  471.454987  468.269989  \
1  2022-10-14  11:38:17  475.109985  472.109985   
2  2022-10-14  11:25:32  476.359985  475.109985   
3  2022-10-14  10:37:16  476.359985  475.109985   
4  2022-10-12  12:47:18  501.450012  501.660004   

                                       Текст новости  
0         Check Out What Whales Are Doing With NOC\n  
1  What 7 Analyst Ratings Have To Say About North...  
2  7 Analysts Have This to Say About Northrop Gru...  
3  Benzinga's Top Ratings Upgrades, Downgrades Fo...  
4  Credit Suisse Picks 7 Aerospace, Defense Stock...  
boeing.csv          Дата     Время     Цена до  Цена после   
0  2022-09-29  15:00:27  124.675903  124.540001  \
1  2022-09-28  14:30:52  133.009995  133.330002   
2  2022-09-28  11:42:56  130.130005  130.570007   
3  2022-09-26  14:05:53  127.320000  126.980003   
4  2022-09-23  11:38:28  131.759995  130.229996   

                                

,Дата,Время,Цена до,Цена после,Текст новости
0,2022-10-21,15:21:15,118.610001,119.364998,S&P 500 Rebounds From 2022 Lows This Week As B...
1,2022-10-21,15:05:00,118.610001,119.364998,What's Going On With Amazon Shares\n
2,2022-10-21,13:55:58,117.800003,118.610001,10 Consumer Discretionary Stocks Whale Activit...
3,2022-10-21,13:53:36,117.800003,118.610001,Amazon To $175? Plus JP Morgan Cuts Price Targ...
4,2022-10-20,14:20:03,116.440002,115.209999,Are All Streaming Services Losing Money? Netfl...
...,...,...,...,...,...
664,2020-12-10,14:47:29,41.845001,41.955002,10 Health Care Stocks With Unusual Options Ale...
665,2020-12-10,11:36:18,41.930000,41.735001,Rehearsal Exposes Gaps In COVID Vaccine Delive...
666,2020-12-10,11:24:03,41.939999,41.930000,"Investors Seem More Focused On Vaccine, Stimul..."
667,2020-12-09,11:29:30,41.520000,41.779999,Allergy Warning Issued For Pfizer-BioNTech COV...


In [19]:
df = df.drop_duplicates(subset=["Текст новости"], keep=False)
df.rename(columns={"Цена до": "price_before", "Цена после": "price_after", "Дата": "date", "Время": "Time", "Текст новости": "news_text"}, inplace=True)

df["absolute_price_difference"] = df["price_after"] - df["price_before"]
df["percentage_price_difference"] = df["absolute_price_difference"] / df["price_before"] * 100
df["price_change_direction"] = np.where(df["absolute_price_difference"] > 0, 1, -1)
df = df[["news_text", "price_change_direction"]].copy()
df.head()

,news_text,price_change_direction
0,Check Out What Whales Are Doing With NOC\n,-1
1,What 7 Analyst Ratings Have To Say About North...,-1
2,7 Analysts Have This to Say About Northrop Gru...,-1
3,"Benzinga's Top Ratings Upgrades, Downgrades Fo...",-1
5,Looking Into Northrop Grumman's Recent Short I...,1


In [20]:
df[df["price_change_direction"] == 1].count()

news_text                 2600
price_change_direction    2600
dtype: int64

In [21]:
df[df["price_change_direction"] == 0].count()

news_text                 0
price_change_direction    0
dtype: int64

In [22]:
X = df["news_text"].values
y = df["price_change_direction"].values

In [23]:
processed_features = []

for sentence in range(len(X)):
    processed_feature = re.sub(r'\W', ' ', str(X[sentence]))

    # remove all single characters
    processed_feature= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)

    # Remove single characters from the start
    processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature)

    # Substituting multiple spaces with single space
    processed_feature = re.sub(r'\s+', ' ', processed_feature, flags=re.I)

    # Removing prefixed 'b'
    processed_feature = re.sub(r'^b\s+', '', processed_feature)

    # Converting to Lowercase
    processed_feature = processed_feature.lower()

    processed_features.append(processed_feature)

In [24]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [25]:
data_words = list(sent_to_words(processed_features))

print(data_words[:10])

[['check', 'out', 'what', 'whales', 'are', 'doing', 'with', 'noc'], ['what', 'analyst', 'ratings', 'have', 'to', 'say', 'about', 'northrop', 'grumman'], ['analysts', 'have', 'this', 'to', 'say', 'about', 'northrop', 'grumman'], ['benzinga', 'top', 'ratings', 'upgrades', 'downgrades', 'for', 'october'], ['looking', 'into', 'northrop', 'grumman', 'recent', 'short', 'interest'], ['this', 'is', 'what', 'whales', 'are', 'betting', 'on', 'northrop', 'grumman'], ['here', 'how', 'much', 'invested', 'in', 'northrop', 'grumman', 'years', 'ago', 'would', 'be', 'worth', 'today'], ['uber', 'nextera', 'are', 'cnbc', 'fast', 'money', 'mentions', 'and', 'this', 'struggling', 'sector', 'is', 'tuesday', 'call', 'of', 'the', 'day'], ['what', 'analyst', 'ratings', 'have', 'to', 'say', 'about', 'northrop', 'grumman'], ['northrop', 'grumman', 'ex', 'dividend', 'date', 'is', 'friday', 'here', 'what', 'you', 'need', 'to', 'know']]


In [26]:
def detokenize(text):
    return TreebankWordDetokenizer().detokenize(text)

In [27]:
data = []
for i in range(len(data_words)):
    data.append(detokenize(data_words[i]))
print(data[:5])

['check out what whales are doing with noc', 'what analyst ratings have to say about northrop grumman', 'analysts have this to say about northrop grumman', 'benzinga top ratings upgrades downgrades for october', 'looking into northrop grumman recent short interest']


In [28]:
labels = np.array(df['price_change_direction'])
y = []
for i in range(len(labels)):
    if labels[i] == -1:
        y.append(0)
    if labels[i] == 1:
        y.append(1)
y = np.array(y)
labels = tf.keras.utils.to_categorical(y, 2, dtype="float32")
del y

In [29]:
len(labels)

5173

In [30]:
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint

max_words = 10000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data)
sequences = tokenizer.texts_to_sequences(data)
news = pad_sequences(sequences, maxlen=max_len)
print(news)
news

[[   0    0    0 ...  422   16 3780]
 [   0    0    0 ...   56  351  352]
 [   0    0    0 ...   56  351  352]
 ...
 [   0    0    0 ...   35   69  401]
 [   0    0    0 ...  164  915  993]
 [   0    0    0 ...   69    2  319]]


array([[   0,    0,    0, ...,  422,   16, 3780],
       [   0,    0,    0, ...,   56,  351,  352],
       [   0,    0,    0, ...,   56,  351,  352],
       ...,
       [   0,    0,    0, ...,   35,   69,  401],
       [   0,    0,    0, ...,  164,  915,  993],
       [   0,    0,    0, ...,   69,    2,  319]], dtype=int32)

In [31]:
print(labels)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [0. 1.]]


In [32]:
#  Split data to train and test sets

X_train, X_test, y_train, y_test = train_test_split(news ,labels, test_size=0.2, shuffle = True, random_state=0)
print (len(X_train),len(X_test),len(y_train),len(y_test))


4138 1035 4138 1035


In [38]:
from keras.layers import BatchNormalization

# Neural network without LSTM

model0 = Sequential()

model0.add(layers.Dense(64, activation="relu"))
model0.add(layers.Dense(32, activation="relu"))
model0.add(layers.Dropout(0.5))
model0.add(layers.Dense(32, activation="relu"))
model0.add(layers.Dropout(0.3))
model0.add(layers.Dense(16, activation="relu"))
model0.add(layers.Dense(2, activation="sigmoid"))

In [39]:
model0.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-6), loss="categorical_crossentropy", metrics=["accuracy"])

checkpoint0 = ModelCheckpoint("best_model0.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1, save_weights_only=False)

history = model0.fit(X_train, y_train,
                     validation_data=(X_test, y_test),
                     verbose=2,
                     epochs=70,
                     batch_size=8,
                     shuffle=True,
                     callbacks=[checkpoint0])

Epoch 1/70

Epoch 1: val_accuracy improved from -inf to 0.48986, saving model to best_model0.hdf5
518/518 - 1s - loss: 22.9328 - accuracy: 0.5039 - val_loss: 1.5195 - val_accuracy: 0.4899 - 1s/epoch - 2ms/step
Epoch 2/70

Epoch 2: val_accuracy did not improve from 0.48986
518/518 - 0s - loss: 1.0242 - accuracy: 0.4957 - val_loss: 0.7185 - val_accuracy: 0.4889 - 493ms/epoch - 951us/step
Epoch 3/70

Epoch 3: val_accuracy did not improve from 0.48986
518/518 - 0s - loss: 0.7454 - accuracy: 0.4981 - val_loss: 0.7064 - val_accuracy: 0.4879 - 494ms/epoch - 954us/step
Epoch 4/70

Epoch 4: val_accuracy improved from 0.48986 to 0.51208, saving model to best_model0.hdf5
518/518 - 1s - loss: 0.7283 - accuracy: 0.5019 - val_loss: 0.6957 - val_accuracy: 0.5121 - 551ms/epoch - 1ms/step
Epoch 5/70

Epoch 5: val_accuracy did not improve from 0.51208
518/518 - 1s - loss: 0.7262 - accuracy: 0.4903 - val_loss: 0.7099 - val_accuracy: 0.4860 - 509ms/epoch - 983us/step
Epoch 6/70

Epoch 6: val_accuracy did 

In [35]:
# Neural network creation
model1 = Sequential()

model1.add(layers.Embedding(max_words, 20))
model1.add(layers.SpatialDropout1D(0.2))
model1.add(layers.LSTM(30, dropout=0.2, recurrent_dropout=0.2))

model1.add(layers.Dense(2, activation='sigmoid'))

In [37]:
model1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-6), loss='categorical_crossentropy', metrics=['accuracy'])

#Implementing model checkpoins to save the best metric and do not lose it on training.
checkpoint1 = ModelCheckpoint("best_model1.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1, save_weights_only=False)

history = model1.fit(X_train, y_train, epochs=70,
                     batch_size=8,
                     verbose=2,
                     validation_data=(X_test, y_test),
                     callbacks=[checkpoint1])

Epoch 1/70

Epoch 1: val_accuracy improved from -inf to 0.56618, saving model to best_model1.hdf5
518/518 - 38s - loss: 0.1320 - accuracy: 0.9280 - val_loss: 2.3035 - val_accuracy: 0.5662 - 38s/epoch - 73ms/step
Epoch 2/70

Epoch 2: val_accuracy improved from 0.56618 to 0.57005, saving model to best_model1.hdf5
518/518 - 35s - loss: 0.1344 - accuracy: 0.9275 - val_loss: 2.0841 - val_accuracy: 0.5700 - 35s/epoch - 68ms/step
Epoch 3/70

Epoch 3: val_accuracy improved from 0.57005 to 0.58164, saving model to best_model1.hdf5
518/518 - 36s - loss: 0.1271 - accuracy: 0.9345 - val_loss: 1.8439 - val_accuracy: 0.5816 - 36s/epoch - 70ms/step
Epoch 4/70

Epoch 4: val_accuracy did not improve from 0.58164
518/518 - 36s - loss: 0.1350 - accuracy: 0.9241 - val_loss: 2.1416 - val_accuracy: 0.5681 - 36s/epoch - 70ms/step
Epoch 5/70

Epoch 5: val_accuracy did not improve from 0.58164
518/518 - 46s - loss: 0.1294 - accuracy: 0.9309 - val_loss: 2.2479 - val_accuracy: 0.5681 - 46s/epoch - 88ms/step
Epoc

In [40]:
model2 = Sequential()
model2.add(layers.Embedding(max_words, 15, input_length=max_len))
model2.add(layers.Bidirectional(layers.LSTM(10,dropout=0.6)))
model2.add(layers.Dense(2, activation='sigmoid'))
model2.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])

2023-05-13 22:03:18.976224: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-13 22:03:18.977470: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-13 22:03:18.978432: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [42]:
checkpoint2 = ModelCheckpoint("best_model2.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history = model2.fit(X_train, y_train, epochs=70,validation_data=(X_test, y_test),callbacks=[checkpoint2])

Epoch 1/70
129/130 [============================>.] - ETA: 0s - loss: 0.4915 - accuracy: 0.7515
Epoch 1: val_accuracy improved from -inf to 0.55652, saving model to best_model2.hdf5
130/130 [==============================] - 6s 45ms/step - loss: 0.4913 - accuracy: 0.7516 - val_loss: 0.7954 - val_accuracy: 0.5565
Epoch 2/70
130/130 [==============================] - ETA: 0s - loss: 0.4729 - accuracy: 0.7629
Epoch 2: val_accuracy improved from 0.55652 to 0.57101, saving model to best_model2.hdf5
130/130 [==============================] - 6s 46ms/step - loss: 0.4729 - accuracy: 0.7629 - val_loss: 0.8135 - val_accuracy: 0.5710
Epoch 3/70
130/130 [==============================] - ETA: 0s - loss: 0.4524 - accuracy: 0.7849
Epoch 3: val_accuracy did not improve from 0.57101
130/130 [==============================] - 6s 45ms/step - loss: 0.4524 - accuracy: 0.7849 - val_loss: 0.8381 - val_accuracy: 0.5401
Epoch 4/70
130/130 [==============================] - ETA: 0s - loss: 0.4475 - accuracy: 0

In [ ]:
best_model = tf.keras.models.load_model("best_model1.hdf5")

In [ ]:
test_loss, test_acc = best_model.evaluate(X_test, y_test, verbose=2)
print('Model accuracy: ',test_acc)